In this notebook, we will look at two parts, **clustering** and **recommender system**.

In the first part, we will go through `KMeans` and `DBSCAN` algorithm to cluster `Starbucks` location geographically. We will use `scikit-learn` for implementation and `matplotlib` for visualisation.

In the second part, we will consider recommender systems with decomposition

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Clustering

First, let's look at the data for clustering

In [ ]:
# Read csv with pandas, index indicates whether the first column is an index
data = pd.read_csv("data/starbucks_locations.csv", index_col=0)
# We don't need NAs, so drop them!
data = data.dropna()

print(data.head())

In [ ]:
# Reduce data for computability. just keep first 200
# data = data[:200]

# or reduce data to just some part of the planet
new_data = data[(data["Longitude"].between(49, 51)) & (data["Latitude"].between(26, 27))]

new_data

Now we can use `scikit-learn` to implement both `k-means` and `DBSCAN`. 

## K-means

In [ ]:
# K-means with 10 neighbours and 500 iterations
kmeans = KMeans(n_clusters=10, 
                max_iter=500)

print('Start K-means: ')
kmeans.fit(data)
print("Cluster centroids: " + str(kmeans.cluster_centers_))

To visualise the results, we need to write a plot function:

In [ ]:
def graphClusters(all_data, labels, centers):
    # open a color map (cmap) which is a collection of colors which you can fetch by index (i.e. an integer)
    cmap = plt.colormaps.get_cmap('tab10')
    
    # add the color as a new column in the dataframe and add the cluster labels
    all_data['color'] = labels
    
    # add the centroids (in case of K-means) to the plot by adding their coordinates  
    # example pyplot use: plt.plot(x, y, 'bo', linestyle='dashed')
    # example pyplot use: plt.plot(x, y,  color='blue', marker='o', linestyle='dashed')
    # more here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html#main-content
    
    which_centroid = 0
    if centers:
        for center in centers:     
            plt.plot(center[0],center[1], marker='o',label='centroid_'+str(which_centroid),
                     markersize=10,color=cmap(which_centroid))
            which_centroid += 1
    
    # add all the data points and use their label number to fetch a certain color
    # index is    
    for ind,row in all_data.iterrows():
        plt.plot(row['Longitude'],row['Latitude'], marker='o', color=cmap(int(row['color'])), markersize=2)
        
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

In [ ]:
# The visulisation requires a long time, please be patient. Or you can try new_data or any other subset of original dataset
# For example, using `new_data` to replace `data`
graphClusters(data,kmeans.labels_,list(kmeans.cluster_centers_))

## Open questions: why is it 10 clusters? How about less or more? Why iteration = 500? Is it possible to have less iterations?

The best tutorial of a Python library is always the official documentation, for example, `K-means`: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

## DBSCAN

In [ ]:
dbscan = DBSCAN(eps=10,
                min_samples=80,
                metric='euclidean')

print('Start DBSCAN: ')
dbscan.fit(data)
print("Labels: "+str(set(dbscan.labels_)))

As we discussed in the lecture, DBSCAN will finally find the outliers. In this case, `-1` means the outlier. So we can revise the plot function to show the outliers.

In [ ]:
def graphClusters_DBSCAN(all_data, labels, centers):
    # open a color map (cmap) which is a collection of colors which you can fetch by index (i.e. an integer)
    cmap = plt.colormaps.get_cmap('tab10')
    
    # add the color as a new column in the dataframe and add the cluster labels
    all_data['color'] = labels
    
    # add the centroids (in case of K-means) to the plot by adding their coordinates  
    # example pyplot use: plt.plot(x, y, 'bo', linestyle='dashed')
    # example pyplot use: plt.plot(x, y,  color='blue', marker='o', linestyle='dashed')
    # more here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html#main-content
    
    which_centroid = 0
    if centers:
        for center in centers:     
            plt.plot(center[0],center[1], marker='o',label='centroid_'+str(which_centroid),
                     markersize=10,color=cmap(which_centroid))
            which_centroid += 1
    
    # add all the data points and use their label number to fetch a certain color
    # index is    
    for ind,row in all_data.iterrows():
        if row['color'] !=-1:
            plt.plot(row['Longitude'],row['Latitude'], marker='o', color=cmap(int(row['color'])), markersize=2)
        else:
                plt.plot(row['Longitude'],row['Latitude'], marker='x', color='black', markersize=5)
        
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

In [ ]:
# The visulisation requires a long time, please be patient. Or you can try new_data or any other subset of original dataset
graphClusters_DBSCAN(data,dbscan.labels_,[])
# the black stars are alll outliers

## Open questions: Can you try other eps and min_sample to improve the results of clustering?

The best tutorial of a Python library is always the official documentation, for example, `DBSCAN`: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

# Recommender system with decomposition

## Non-Negative Matrix Factorization (NMF)

- A matrix factorization technique where a matrix `M` is factorized into two matrices \(U\) and \(V\), with the constraint that all three matrices have no negative elements.
- Used to discover latent features underlying the interactions between two types of entities (e.g., users and items).

Goal:
- Captures complex user preferences and item characteristics.
- Helps in providing personalized recommendations.
- Offers insights into latent factors driving user-item interactions.

Therefore we can consider decomposition. 
We convert the utility matrix $M (r \times m)$ into $M' = UV^T \approx M$  with $U (r \times l)$ and $V^T (l \times m)$.

In [ ]:
from sklearn.decomposition import NMF

loading the dataset

In [ ]:
# load data
ratings = pd.read_csv('data/ratings.csv')

# sample dataset
# be careful, once again a very heavy operation
ratings = ratings[:1000]

print(ratings.head())

# print some information
noMovies = len(ratings['movieId'].unique())
noUsers = len(ratings['userId'].unique())
print(str(noMovies)+" from "+str(noUsers)+' users')

Do some pre-processing before we carefully look at it.

In [ ]:
# create empty utility matrix
utility = np.zeros(shape=(noUsers,noMovies))

# store movieIds as indices to use in utility matrix
movieIds = {}
midi = 0
for value in ratings['movieId'].unique():
    movieIds[value]=midi
    midi = midi + 1

# populate utility matrix
for index, line in ratings.iterrows():
    uid = int(line['userId'])-1
    mid = movieIds[line['movieId']]
    rating = line['rating']
    utility[uid,mid]=rating

- utility: The original user-item unitily matrix.
- U: The user feature matrix.
- V: The item feature matrix.

Now let's do the matrix factorisation with `scikit-learn`:

In [ ]:
decomposition = NMF(n_components=50, init='random', random_state=0, max_iter=500)
U = decomposition.fit_transform(utility)
V_T = decomposition.components_

In the matrix `U`: 
- Each row represents a user.
- Each column represents a latent feature.
- Elements of \(U\) indicate the association strength between users and latent features.
- In our case, latent features might include genres or themes like action, romance, or sci-fi.

In the matrix `V`
- Each row represents a latent feature.
- Each column represents an item.
- Elements of \(V\) indicate how strongly an item is associated with each latent feature.
- This helps in understanding what characteristics of items appeal to users.

In [ ]:
print('Shape of U (#reviewers x #latent factors): ', np.shape(U))
print('Shape of V_T (#latent factors x #movies): ', np.shape(V_T))

Now we can calculate $M'$ and compare how close `M'` and `M`

In [ ]:
M_ = np.dot(U, V_T)
print(np.shape(M_))

We can see how more dimensions provide a closer approximation of the original matrix:

In [ ]:
# Please be patient, this code is very slow (how slow depending on the max_iter and n_comp).
# You can try some small values on range() or max_iter
for n_comp in range(20,100,10):
    decomposition = NMF(n_components=n_comp, init='random', random_state=1, max_iter=200)
    U = decomposition.fit_transform(utility)
    V_T = decomposition.components_
    M_ = np.dot(U, V_T)
    
    # calculate difference between both matrices
    diff = utility-M_
    print(f"for {n_comp} components,\tdifference was {np.sum(diff)}")
    
print("Completed!") # this will run for a while, up to a minute or two.


more reading here: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

What do you see? is the difference between matrixes getting larger or smaller? is that good?

## Open Questions: Can you store all `diff` and plot a figure, the number of components vs diff, and observe the pattern?